# Финальное задание модуля PYTHON-13 #
## (Анализ и визуализация данных) ##
Подробное описание содержится в файле [**`README.md`**](https://github.com/stalkspectrum/sf-ds-course-all/blob/master/01-16-PYTHON-13/P13_Task/README.md).

In [2]:
import pandas as pd
bank_df = pd.read_csv('churn.csv', sep=',')
display(bank_df)
display(bank_df.info())

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


None

----

### Задание 9.1. ###
Каково соотношение ушедших и лояльных клиентов? Покажите это на графике и дайте комментарий по соотношению.

----

### Задание 9.2. ###
Постройте график, показывающий распределение баланса пользователей, у которых на счету больше **2500** долларов. Опишите распределение и сделайте выводы.

----

### Задание 9.3. ###
Посмотрите на распределение баланса клиента в разрезе признака оттока. Как различаются суммы на накопительном счёте ушедших и лояльных клиентов? Подумайте и напишите, с чем это может быть связано, что может не устраивать ушедших клиентов в банке.

----

### Задание 9.4. ###
Посмотрите на распределение возраста в разрезе признака оттока. В какой группе больше потенциальных выбросов? На какую возрастную категорию клиентов стоит обратить внимание банку?

----

### Задание 9.5. ###
Постройте график, который показывает взаимосвязь кредитного рейтинга клиента и его предполагаемой зарплаты. Добавьте расцветку по признаку оттока клиентов. Какова взаимосвязь между признаками? Если не видите явной взаимосвязи, укажите это.

----

### Задание 9.6. ###
Кто чаще уходит, мужчины или женщины? Постройте график, который иллюстрирует это.

***Подсказка***

Процент ушедших клиентов в каждой группе можно рассчитать как среднее по столбцу **`Exited`** (так как **1** — это ушедшие клиенты, а **0** — лояльные, среднее арифметическое по столбцу обозначает долю ушедших клиентов).

----

### Задание 9.7. ###
Как отток клиентов зависит от числа приобретённых у банка услуг? Для ответа на этот вопрос постройте многоуровневую столбчатую диаграмму.

----

### Задание 9.8. ###
Как влияет наличие статуса активного клиента на отток клиентов? Постройте диаграмму, иллюстрирующую это. Что бы вы предложили банку, чтобы уменьшить отток клиентов среди неактивных?

----

### Задание 9.9. ###
В какой стране доля ушедших клиентов больше? Постройте тепловую картограмму, которая покажет это соотношение на карте мира. Предположите, с чем это может быть связано.

----

### Задание 9.10. ###
Переведите числовой признак **`CreditScore`** в категориальный. Для этого воспользуйтесь функцией `get_credit_score_cat()`, которая приведена ниже. Примените её к столбцу **`CreditScore`** и создайте новый признак **`CreditScoreCat`** — категории кредитного рейтинга.

Постройте сводную таблицу, строками которой являются категории кредитного рейтинга (**`CreditScoreCat`**), а столбцами — количество лет, в течение которых клиент пользуется услугами банка (**`Tenure`**). В ячейках сводной таблицы должно находиться среднее по признаку оттока (**`Exited`**) — доля ушедших пользователей.

На основе полученной сводной таблицы постройте тепловую карту с аннотацией. Найдите на тепловой карте категории клиентов, которые уходят чаще всего.